In [2]:
from jinja2 import Template
template = Template(u'Greetings, {{ name }}!')
template.render(name='Mr. Praline')

u'Greetings, Mr. Praline!'

In [20]:
from jinja2 import Environment, PackageLoader, FileSystemLoader
import csv
import json
from datetime import datetime
from itertools import groupby
from operator import itemgetter

def dataset(path, include):
    column = 'Average income per tax unit'
    
    with open(path, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        key = itemgetter('Country')
        
        # Use groupby: memory efficient collection by country
        for key, values in groupby(reader, key=key):
            # Only yield countries that are included
            if key in include:
                yield key, [(int(value['Year']), 
                                 float(value[column]))
                           for value in values if value[column]]
                
def extract_years(data):
    for country in data:
        for value in country[1]:
            yield value[0]
            
def extract_series(data, years):
    for country, cdata in data:
        cdata  = dict(cdata)
        series = [cdata[year] if year in cdata else None for year in years]
        yield {
            'name': country,
            'data': series,
        }
        
def write(context):
    path = "report-%s.html" % datetime.now().strftime("%Y%m%d")
    #jinjaenv = Environment(loader=PackageLoader('reporting', 'templates'))
    # 'templates' should be the path to the templates folder
    # as written, it is assumed to be in the current directory
    jinjaenv = Environment(loader = FileSystemLoader('code/templates'))
    template = jinjaenv.get_template('report.html')
    template.stream(context).dump(path)
    
def main(source):
    include = ("United States", "France", "Italy", 
               "Germany", "South Africa", "New Zealand")
    
    data = list(dataset(source, include))
    years = set(extract_years(data))
    
    context = {
        'title': "Average Income per Family, %i - %i" %
        (min(years), max(years)),
        'years': json.dumps(list(years)),
        'countries': [v[0] for v in data],
        'series': json.dumps(list(extract_series(data, years)))
    }
    
    write(context)
    
if __name__ == "__main__":
    source = './data/income_dist.csv'
    main(source)

In [15]:
print template.stream()